# Dynamic_Quantization

> We will be implementing Dynamic_Quantization model compression here

In [1]:
#| default_exp quantization_fastai

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from fastai.vision.all import *  # Import necessary libraries
import os
import sys
import torch
import time
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
import logging
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm, trange
import torch.quantization
sys.path.append('/root/hsi-compression/models/')
from cae1d import ConvolutionalAutoencoder1D


In [4]:

#| export

filepath = "/root/hsi-compression/results/weights/cae1d_8bpppc.pth.tar"

def load_model_with_weights(filepath, quantized=False):
    model = ConvolutionalAutoencoder1D()
    if quantized:
        qconfig = torch.quantization.get_default_qconfig('fbgemm')
        model.qconfig = qconfig
        model = torch.quantization.prepare(model, inplace=False)
    
    checkpoint = torch.load(filepath, map_location='cpu', weights_only=True)  # Ensure loading to CPU
    if quantized:
        model = torch.quantization.convert(model, inplace=False)  # Convert to quantized version

    model_state_dict = {k: v for k, v in checkpoint.items() if 'activation_post_process' not in k}
    model.load_state_dict(model_state_dict, strict=False)
    model.to('cpu')  # Explicitly move the model to CPU
    model.eval()
    return model






In [5]:
# Append the directory containing the cae1d.py file, not the file itself

# Setup logging to help with debugging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Base directory where the .npy files are stored
base_directory = '/root/hsi-compression/datasets/hyspecnet-11k/patches/'

# Function to load paths from a CSV file without headers
def load_paths(csv_file):
    df = pd.read_csv(csv_file, header=None)
    file_paths = [os.path.join(base_directory, x.strip()) for x in df[0]]
    return file_paths

class NPYDataset(Dataset):
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        sample = np.load(file_path)
        if self.transform:
            sample = self.transform(sample)
        sample = torch.from_numpy(sample).float().to('cpu')  # Move data to CPU
        return sample, sample

# Apply any necessary transformations directly on the CPU
def transform_sample(sample):
    return (sample - np.mean(sample)) / np.std(sample)

csv_file_path = '/root/hsi-compression/datasets/hyspecnet-11k/splits/easy/test.csv'
file_paths = load_paths(csv_file_path)

dataset = NPYDataset(file_paths, transform=transform_sample)
print("Dataset initialized successfully.")



Dataset initialized successfully.


In [6]:
#| export

def apply_dynamic_quantization(model, save_path="/root/hsi-compression/compressed_model/quantized_model.pth"):
    
    # Apply dynamic quantization to Conv1d layers
    quantized_model = torch.quantization.quantize_dynamic(
        model,
        {nn.Conv1d},  # Target Conv1d layers
        dtype=torch.qint8  # Quantize using 8-bit integers
    )
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Save the quantized model
    torch.save(quantized_model.state_dict(), save_path)
    print(f"Quantized model saved at: {save_path}")
    
    return quantized_model




In [7]:
#| export

def evaluate_model(model, dataloader):
    model.eval()
    losses = []
    criterion = torch.nn.MSELoss()
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating model", leave=False):
            inputs, labels = inputs.to('cpu'), labels.to('cpu')  # Ensure tensors are on CPU
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
    mean_loss = sum(losses) / len(losses)
    return mean_loss


In [8]:
#| export

def evaluate_ssim(model, dataloader):
    model.eval()
    ssim_scores = []
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)
            output_np = output.cpu().detach().numpy()
            target_np = target.cpu().detach().numpy()
            for o, t in zip(output_np, target_np):
                score = ssim(o, t, data_range=t.max() - t.min())
                ssim_scores.append(score)
    average_ssim = np.mean(ssim_scores)
    return average_ssim


In [9]:
#| export

def evaluate_latency(model, dataloader, num_iterations=100):
    start_time = time.time()
    model.eval()
    with torch.no_grad():
        for i, (data, _) in enumerate(dataloader):
            if i >= num_iterations:
                break
            _ = model(data)
    end_time = time.time()
    total_time = end_time - start_time
    average_time_per_batch = total_time / num_iterations
    return average_time_per_batch


In [10]:

# Append the directory containing the cae1d.py file, not the file itself
sys.path.append('/root/hsi-compression/models/')

from cae1d import ConvolutionalAutoencoder1D

# Load your model
filepath = "/root/hsi-compression/results/weights/cae1d_8bpppc.pth.tar"
model = load_model_with_weights(filepath)  # Make sure this function is correctly defined

subset_size = int(len(dataset) * 0.5)
indices = torch.randperm(len(dataset))[:subset_size]
subset = Subset(dataset, indices)
dataloader = DataLoader(subset, batch_size=16, shuffle=True, num_workers=0)
print("DataLoader initialized successfully.")

# Now evaluate the model
print("Original Model Accuracy:", evaluate_model(model, dataloader))

# If you have a quantized model
quantized_model = apply_dynamic_quantization(model)  # Ensure this function is correctly defined and imported
print("Quantized Model Accuracy:", evaluate_model(quantized_model, dataloader))


original_ssim = evaluate_ssim(model, dataloader)
quantized_ssim = evaluate_ssim(quantized_model, dataloader)
print(f"Original SSIM: {original_ssim:.4f}, Quantized SSIM: {quantized_ssim:.4f}")


# Measure latency
original_latency = evaluate_latency(model, dataloader)
quantized_latency = evaluate_latency(quantized_model, dataloader)
print(f"Original Model Latency: {original_latency:.4f} seconds per batch")
print(f"Quantized Model Latency: {quantized_latency:.4f} seconds per batch")

# size evaluation

def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)

def get_model_size(filepath):
    size_bytes = os.path.getsize(filepath)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Paths for original and quantized models
original_model_path = "/root/hsi-compression/results/weights/cae1d_8bpppc.pth.tar"
quantized_model_path = "/root/hsi-compression/compressed_model/quantized_model.pth"

# Assuming 'model' is your original loaded model
save_model(model, original_model_path)

save_model(quantized_model, quantized_model_path)  

# Calculate and print the model sizes
original_size = get_model_size(original_model_path)
quantized_size = get_model_size(quantized_model_path)  
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Quantized Model Size: {quantized_size:.2f} MB")



DataLoader initialized successfully.


Original Model Accuracy: 0.6015384942293167
Quantized model saved at: /root/hsi-compression/compressed_model/quantized_model.pth


Quantized Model Accuracy: 0.6033445373177528
Original SSIM: 0.3540, Quantized SSIM: 0.3540
Original Model Latency: 1.6883 seconds per batch
Quantized Model Latency: 1.6928 seconds per batch
Original Model Size: 0.22 MB
Quantized Model Size: 0.22 MB


In [11]:
#| export
def foo(): pass

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()